# What's in this notebook?
- Code for scraping Trader Joes site for all their addresses
- Loading TJ's addresses into a nice readable dataframe
- Code for gather census info on each address' area (by census group block)
- Loading census info into a nice readable data frame

## Scraping TJ's Site

In [23]:
import requests 
from bs4 import BeautifulSoup
import time
url = "https://locations.traderjoes.com"

home = requests.get(url)

In [18]:
# home page gets you to all the links to the locations by state
state_locs = [] # hold all the urls for tj locations by state
soup = BeautifulSoup(home.content, 'html5lib') 
for div in soup.findAll('div', attrs = {'class':'itemlist'}):
    state_locs.append(div.a['href'])

In [24]:
# now we need to go by city within each state to get the tjs locations
locations = []
for state_url in state_locs:
    state = requests.get(state_url)
    soup = BeautifulSoup(state.content, 'html5lib') 
    for div in soup.findAll('div', attrs = {'class': 'itemlist'}):
        locations.append(div.a['href'])
    time.sleep(1)

locations

['https://locations.traderjoes.com/al/birmingham/',
 'https://locations.traderjoes.com/az/gilbert/',
 'https://locations.traderjoes.com/az/glendale/',
 'https://locations.traderjoes.com/az/mesa/',
 'https://locations.traderjoes.com/az/oro-valley/',
 'https://locations.traderjoes.com/az/phoenix/',
 'https://locations.traderjoes.com/az/prescott/',
 'https://locations.traderjoes.com/az/scottsdale/',
 'https://locations.traderjoes.com/az/surprise/',
 'https://locations.traderjoes.com/az/tempe/',
 'https://locations.traderjoes.com/az/tucson/',
 'https://locations.traderjoes.com/ca/agoura-hills/',
 'https://locations.traderjoes.com/ca/alameda/',
 'https://locations.traderjoes.com/ca/aliso-viejo/',
 'https://locations.traderjoes.com/ca/arroyo-grande/',
 'https://locations.traderjoes.com/ca/bakersfield/',
 'https://locations.traderjoes.com/ca/berkeley/',
 'https://locations.traderjoes.com/ca/brea/',
 'https://locations.traderjoes.com/ca/brentwood/',
 'https://locations.traderjoes.com/ca/burban

In [56]:
# now get the addresses for each store in each location in the city
addresses = []
for location in locations:
    loc = requests.get(location)
    soup = BeautifulSoup(loc.content, 'html5lib')
    for loc in soup.findAll('div', attrs = {'class': 'address-left'}):
        address = []
        for x in loc.findAll('span')[1:5]:
            address.append(x.text)
        addresses.append(address)
    time.sleep(.5)

addresses

[['205 Summit Blvd, Suite 100', 'Birmingham', 'AL', '35243'],
 ['1779 E. Williams Field Rd.', 'Gilbert', 'AZ', '85295'],
 ['7720 West Bell Rd', 'Glendale', 'AZ', '85308'],
 ['2050 E Baseline Rd', 'Mesa', 'AZ', '85204'],
 ['7912 N Oracle', 'Oro Valley', 'AZ', '85704'],
 ['4025 E Chandler Blvd', 'Phoenix', 'AZ', '85048'],
 ['4726 East Shea Blvd', 'Phoenix', 'AZ', '85028'],
 ['4821 N 20th St', 'Phoenix', 'AZ', '85016'],
 ['252 N Lee Blvd', 'Prescott', 'AZ', '86303'],
 ['7555 E Frank Lloyd Wright', 'Scottsdale', 'AZ', '85260'],
 ['6202 N Scottsdale Rd', 'Scottsdale', 'AZ', '85253'],
 ['14095 W Grand Ave', 'Surprise', 'AZ', '85374'],
 ['6460 S McClintock Dr', 'Tempe', 'AZ', '85283'],
 ['1101 N Wilmot Rd', 'Tucson', 'AZ', '85712'],
 ['4209 N Campbell Ave', 'Tucson', 'AZ', '85719'],
 ['4766 E Grant Rd', 'Tucson', 'AZ', '85712'],
 ['28941 Canwood St', 'Agoura Hills', 'CA', '91301'],
 ['2217 South Shore Center', 'Alameda', 'CA', '94501'],
 ['26541 Aliso Creek Rd', 'Aliso Viejo', 'CA', '92656'],

## Loading TJ's Info Into a Dataframe

In [20]:
# we should probably turn this into something friendly -- we'll make it a dataframe
import pandas as pd

df = pd.DataFrame(data=addresses, columns=['street', 'city', 'state', 'zip'])

In [3]:
# lets pickle our work so we don't have to do it again 
import pickle
pickle.dump(df, open( "tj-addresses.pickle", "wb" ) )

In [70]:
df

,street,city,state,zip
0,"205 Summit Blvd, Suite 100",Birmingham,AL,35243
1,1779 E. Williams Field Rd.,Gilbert,AZ,85295
2,7720 West Bell Rd,Glendale,AZ,85308
3,2050 E Baseline Rd,Mesa,AZ,85204
4,7912 N Oracle,Oro Valley,AZ,85704
5,4025 E Chandler Blvd,Phoenix,AZ,85048
6,4726 East Shea Blvd,Phoenix,AZ,85028
7,4821 N 20th St,Phoenix,AZ,85016
8,252 N Lee Blvd,Prescott,AZ,86303
9,7555 E Frank Lloyd Wright,Scottsdale,AZ,85260


In [1]:
# there's aren't any repeats, right? Nope -- 481 stores, all unique.
len(df['street'].unique())

NameError: name 'df' is not defined

## Gathering Census Info - First Try (feel free to skip this section; it does not have a happy ending)
Webscraping the census website using selenium and beautiful soup (did not go well, census site is not friendly)

In [78]:
pip install selenium 

    100% |████████████████████████████████| 911kB 6.5MB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [87]:
# selenium attempt

from selenium import webdriver

EXE_PATH = r'/Users/hannah/Downloads/chromedriver'
driver = webdriver.Chrome(executable_path=EXE_PATH)
driver.get('https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml')

search = driver.find_elements_by_id('cfsearchtextboxmain')[0]  

# find_elements will give us the list of all elements with id as subjectInput 
search.send_keys(35243) # zipcode goes here                   
time.sleep(2)
# hit enter twice to advance to the results
search.send_keys(Keys.ENTER)
search.send_keys(Keys.ENTER)

# find the population census info
for element in driver.find_elements_by_tag_name('a'):
    if element.text == 'General Population and Housing Characteristics (Population, Age, Sex, Race, Households and Housing, ...)':
        element.click()
        
driver.find_element_by_id('data')

In [172]:
# beautiful soup attempt
import requests
from   bs4 import BeautifulSoup

#how to get the population data, as per https://kaijento.github.io/2017/05/14/web-scraping-factfinder.census.gov/

zipcodes = ['11375']

base   = 'https://factfinder.census.gov/'
report = base + 'bkmk/table/1.0/en/DEC/10_DP/DPDP1/8600000US'
render = base + 'tablerestful/tableServices/renderProductData'

with requests.session() as s:
    s.headers['user-agent'] = 'Chrome/76'

    for zipcode in zipcodes:
        s.get(report + zipcode)
        r = s.get(render)
        print(r)
        html = r.json()['ProductData']['productDataTable']
        soup = BeautifulSoup(html, 'html5lib')

<Response [200]>


In [174]:
r.json()

{'ProductData': {'displayID': 'DP-1',
  'displayLabel': 'Profile of General Population and Housing Characteristics: 2010',
  'productDataset': '2010 Demographic Profile Data',
  'eusbreadcrumb': '<div id="pageinstr">\n<span id="pagetitle">Advanced Search</span> - <span id="pagedescription">Search all data in American FactFinder</span>\n</div>\n<div id="steps" class="AS">\n<div class="step completed" onclick="javascript:processTransition(\'datafinder\');"  title="Search all data in American FactFinder">\n<span>1</span> Advanced Search\n</div>\n<div class="step activelaststep" title="Table Viewer">\n<span>2</span> Table Viewer\n</div>\n</div>',
  'currentContext': 'datafinder',
  'breadcrumbTitle': 'Advanced Search',
  'currentContextURI': '/faces/nav/jsf/pages/searchresults.xhtml?refresh=t',
  'backToBreadcrumbTitle': 'Advanced Search',
  'universe': '',
  'tableToolsAvailable': 'true',
  'tableToolsEnabled': 'false',
  'mappable': 'false',
  'statsigSupported': 'false',
  'statsigEnabl

In [199]:
# for x in soup.findAll('table',attrs = {'id': 'data'} ):
#     print(x.text)
print(soup.findAll('table',attrs = {'id': 'data'})[0])

<table class="stat-tbl" id="data"><thead><tr class="h"><th class="metastub left right regular top bottom br-edge" colspan="1" id="pc1" rowspan="1">Subject</th><th class="L0 boxhead label top bottom left regular br-edge" colspan="1" id="c1">Number</th><th class="L0 boxhead label top bottom right regular br-edge" colspan="1" id="c2">Percent</th></tr>
</thead>
<tbody>
<tr class="h"><th class="label L0 regular top left stub br-edge" colspan="1" headers="pc1" id="r1">SEX AND AGE</th><td class="field left top" headers=""> </td><td class="field right top" headers=""> </td></tr>
<tr class="h stripe"><th class="label regular left L1 stub br-edge" colspan="1" headers="pc1" id="r2">Total population</th><td class="field left" headers="c1 r1 r2">68,733</td><td class="field right" headers="c2 r1 r2">100.0</td></tr>
<tr class="h"><th class="label regular L2 left stub br-edge" colspan="1" headers="pc1" id="r3">Under 5 years</th><td class="field left" headers="c1 r1 r2 r3">3,853</td><td class="field ri

In [175]:
# can I replicate this for the education data??
base   = 'https://factfinder.census.gov/'
report = base + '/bkmk/table/1.0/en/ACS/17_5YR/S1501/8600000US35243'
render = base + 'tablerestful/tableServices/renderProductData'

zipcodes = ['11375']

with requests.session() as s:
    s.headers['user-agent'] = 'Chrome/76'

    for zipcode in zipcodes:
        s.get(report + zipcode)
        r = s.get(render)
        
        html = r.json()

## Census Data - Found Online, Already Scraped!
So, it turns out we don't need to scrape the census data! Hooray! The data (from a site called safegraph) is on census block groups, so all that's left to do is figure out what census block group each TJ address is in. The nice thing about the data being grouped by census block group is that census block groups are designed to be homogenous, so we will likely get a good picture of the area that the TJ's was going for when we look at the data retreived from the census block group it's in. 

We have the addresses, so we can use the Census Bureau's GEOID lookup (https://geocoding.geo.census.gov/geocoder) to find the census block group. Unfortunately, their bulk address lookup is glitchy and doesn't always find addresses that are their system, so we have to scrape this info. Luckily, they save the groups from the 2010 census, so we're not going to see weird changes because of new roads that may have changed census block groups since then.

GEOIDs contain the state, country, tract, block group, and block IDs, represented as one long string of digits (in that order). If we take the first 12 digits (going from left to right), that should give us the census block group. (more info on GEOIDs: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html) For example, my college's GEOID is: 360550038021000 -- 36 represents the state (NY), 055 represents the county (Monroe), 003802 represents the census tract, and 1000 represents the census block (1 represents the census block group -- all census blocks in a census block group will have the same first digit in their 4 digit census block, and all the numbers before that first digit will be the same). So, if I wanted the census block group data for U of R, I would look up the first 12 digits -- 360550038021 -- and have my results. 

So, there are 2 things to do:

1. Get the addresses as a CSV with 'columns' street, city, state, and zip
2. Get the GEOID from the census site

Then we can join each TJ's store with its area's demographic/educational/etc. information on census block group. We'll do that part in another notebook, though, since it's not specific to collecting the data (but rather, cleaning it and making it into something actually useful).

In [ ]:
# interesting data:
# B15003e1 - B15003m9 (education data)
# B19001e1 - B19001e9 (household income data -- bucketed)
# B19049e1 - B19049e5 (household income data -- median)
# B25075e1 - B25075e9 (housing value -- owner occupied, bucketed)
# B25085e1 - B25085e9 (housing asking prices)
# B02001e1 - B02001e9 (race info)
# B03002e10 - B03002e9 (race info + hispanic or latino)

In [314]:
import time
geoids = []
for i in range(len(addresses)):
    # get link by putting in address info
    link = f'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?address={addresses.iloc[i,0]}%2C+{addresses.iloc[i,1]}%2C+{addresses.iloc[i,2]}%2C+{addresses.iloc[i,3]}&benchmark=9&vintage=910'
    # get page contents
    page = requests.get(link)
    # get page contents
    soup = BeautifulSoup(page.content, 'html5lib')
    geoids.append((addresses.iloc[i,0], re.findall('GEOID:\s[0-9]{12}', soup.text)))
    time.sleep(.35)
    # for my sanity, let me know where we are every 15 addresses
    if i%15 == 0:
        print(i)

0
15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
480
495


In [315]:
geoids

[('205 Summit Blvd', []),
 ('1779 E. Williams Field Rd.', []),
 ('7720 West Bell Rd', ['GEOID: 040136177003']),
 ('2050 E Baseline Rd', ['GEOID: 040134225034']),
 ('7912 N Oracle', ['GEOID: 040190047132']),
 ('4025 E Chandler Blvd', ['GEOID: 040131167122']),
 ('4726 East Shea Blvd', ['GEOID: 040131032081']),
 ('4821 N 20th St', ['GEOID: 040131085024']),
 ('252 N Lee Blvd', ['GEOID: 040250008021']),
 ('7555 E Frank Lloyd Wright', ['GEOID: 040132168161']),
 ('6202 N Scottsdale Rd', ['GEOID: 040132169012']),
 ('14095 W Grand Ave', []),
 ('6460 S McClintock Dr', ['GEOID: 040133199052']),
 ('1101 N Wilmot Rd', ['GEOID: 040190030023']),
 ('4209 N Campbell Ave', ['GEOID: 040190027011']),
 ('4766 E Grant Rd', ['GEOID: 040190029043']),
 ('28941 Canwood St', []),
 ('2217 South Shore Center', []),
 ('26541 Aliso Creek Rd', ['GEOID: 060590626371']),
 ('955 Rancho Pkwy', ['GEOID: 060790118003']),
 ('8200 Stockdale Hwy', ['GEOID: 060290028061']),
 ('1885 University Ave', ['GEOID: 060014223003']),
 (

In [318]:
geoid_df = pd.DataFrame(geoids, columns=['street', 'geoid'])

In [342]:
geoid_df.geoid = geoid_df.geoid.apply(lambda x: 'NaN' if len(x) == 0 else x)

In [345]:
geoid_df[geoid_df.geoid == 'NaN']

,street,geoid
0,205 Summit Blvd,NaN
1,1779 E. Williams Field Rd.,NaN
11,14095 W Grand Ave,NaN
16,28941 Canwood St,NaN
17,2217 South Shore Center,NaN
22,2500 E Imperial Hwy,NaN
23,5451 Lone Tree Way,NaN
28,3555 Clares St #D,NaN
29,2629 Gateway Rd,NaN
36,878 Eastlake Pkwy,NaN


Okay, 85 TJ's didn't get matched to a GEOID...What to do? When I look some of them up, it seems like there's something weird going on -- it may be that my addresses aren't entered perfectly, since when I look these addresses up, they _do_ come up in the census look up. I'll do a left join on these missing rows and the addresses to see if I notice anything strange. 

In [351]:
missing = geoid_df[geoid_df.geoid == 'NaN'].merge(addresses, how='left')[['street', 'city', 'state', 'zip']]

Well, I didn't notice anything strange, so maybe I'll try a different site -- the FCC has an API that will give you the census block based on the latitude and longitude, and geopy is a lovely library that will give you the latitude and longitude of an address. Let's hope this works!

In [350]:
pip install geopy

    100% |████████████████████████████████| 102kB 3.1MB/s a 0:00:01
  Stored in directory: /Users/hannah/Library/Caches/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
Note: you may need to restart the kernel to use updated packages.


In [388]:
import json
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="student project")
found_geoids = []

for i in range(len(missing)):
    loc = geolocator.geocode(f'{missing.iloc[i,0]}, {missing.iloc[i,1]}, {missing.iloc[i,2]}, {missing.iloc[i,3]}')
    try:
        info = requests.get(f"https://geo.fcc.gov/api/census/block/find?latitude={loc.latitude}&longitude={loc.longitude}&format=json")
        info = json.loads(info.content)
        found_geoids.append((missing.iloc[i, 0], info['Block']['FIPS'][:12]))
    except AttributeError:
        found_geoids.append((missing.iloc[i, 0], 'NaN'))
    time.sleep(.35)

In [392]:
pd.DataFrame(found_geoids)[pd.DataFrame(found_geoids)[1] == 'NaN']

,0,1
18,8086 E Pacific Coast Hwy,NaN
20,1482 El Camino Real,NaN
24,5353 Almaden Expressway #J-38,NaN
26,2300 Wilshire Blvd #101,NaN
27,301 MC Lellan Dr,NaN
30,1851 S Federal Highway #500,NaN
31,4180 S 3rd St,NaN
32,10600 Tamiami Trail N,NaN
33,2877 South State Rd 7,NaN
34,5185 Peachtree Pkwy,NaN


Let's see if this a geopy issue or an FCC API issue. We'll start by joining the dataframes for the missing addresses, like we did before, and see if there's anything weird going on with those addresses.

In [395]:
missing2 = pd.DataFrame(found_geoids)[pd.DataFrame(found_geoids)[1] == 'NaN'].merge(addresses, how='left', 
                                                                                    left_on=0, right_on='street')[['street', 'city', 'state', 'zip']]

In [396]:
missing2

,street,city,state,zip
0,8086 E Pacific Coast Hwy,Newport Beach,CA,92657
1,1482 El Camino Real,San Carlos,CA,94070
2,5353 Almaden Expressway #J-38,San Jose,CA,95118
3,2300 Wilshire Blvd #101,Santa Monica,CA,90403
4,301 MC Lellan Dr,South San Francisco,CA,94080
5,1851 S Federal Highway #500,Delray Beach,FL,33483
6,4180 S 3rd St,Jacksonville Beach,FL,32250
7,10600 Tamiami Trail N,Naples,FL,34108
8,2877 South State Rd 7,Wellington,FL,33414
9,5185 Peachtree Pkwy,Norcross,GA,30092


In [413]:
# get the lat and longs for these -- maybe geopy is the issue?
for i in range(len(missing2)):  
    loc = geolocator.geocode(f'{missing2.iloc[i,0]}, {missing2.iloc[i,1]}, {missing2.iloc[i,2]}, {missing2.iloc[i,3]}')
    try:
        print(loc.latitude, loc.longitude) 
    except AttributeError:
        print('error')

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


Yep, it's the issue.